In [5]:
#!wget -P ds/ https://github.com/openai/CLIP/raw/main/clip/bpe_simple_vocab_16e6.txt.gz
#!wget -P clip/ https://github.com/openai/CLIP/raw/main/clip/bpe_simple_vocab_16e6.txt.gz
#!pip install pip install salesforce-lavis

--2024-03-27 11:44:46--  https://github.com/openai/CLIP/raw/main/clip/bpe_simple_vocab_16e6.txt.gz
proxy.uec.ac.jp (proxy.uec.ac.jp) をDNSに問いあわせています... 130.153.8.47
proxy.uec.ac.jp (proxy.uec.ac.jp)|130.153.8.47|:8080 に接続しています... 接続しました。
Proxy による接続要求を送信しました、応答を待っています... 302 Found
場所: https://raw.githubusercontent.com/openai/CLIP/main/clip/bpe_simple_vocab_16e6.txt.gz [続く]
--2024-03-27 11:44:46--  https://raw.githubusercontent.com/openai/CLIP/main/clip/bpe_simple_vocab_16e6.txt.gz
proxy.uec.ac.jp (proxy.uec.ac.jp)|130.153.8.47|:8080 に接続しています... 接続しました。
Proxy による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1356917 (1.3M) [application/octet-stream]
`clip/bpe_simple_vocab_16e6.txt.gz' に保存中

bpe_simple_vocab_16 100%[===================>]   1.29M  --.-KB/s    in 0.1s    

2024-03-27 11:44:47 (11.0 MB/s) - `clip/bpe_simple_vocab_16e6.txt.gz' へ保存完了 [1356917/1356917]



In [6]:
import os

from os.path import join as ospj
from os.path import expanduser
from munch import Munch as mch
import numpy as np

from ds import prepare_coco_dataloaders, prepare_flickr_dataloaders, prepare_cub_dataloaders, prepare_flo_dataloaders

from utils import *
from networks import *
from train_probVLM import *

import matplotlib.pyplot as plt

/home/nakat/anaconda3/envs/ProbVLM/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853668/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
100%|██████████████████████████████████████| 1.97G/1.97G [01:17<00:00, 27.2MB/s]


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

In [27]:
dataset = 'coco' # coco or flickr
data_dir = ospj('../../datasets', dataset) # e.g. ospj(expanduser('~'), 'Documents', 'jm', 'data', dataset)
dataloader_config = mch({
    'batch_size': 64,
    'random_erasing_prob': 0.,
    'traindata_shuffle': True
})
loaders,vocab = load_data_loader(dataset, data_dir, dataloader_config)
cub_train_loader, cub_valid_loader, cub_test_loader = loaders['train'], loaders['val'], loaders['test']

prepare cub dataset with 150 classes


FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/CUB/text_c10/100.Brown_Pelican/Brown_Pelican_0002_94773.txt'

In [ ]:
# clip_net = load_model('cuda')
CLIP_Net = load_model(device='cuda', model_path=None)
ProbVLM_Net = BayesCap_for_CLIP(
    inp_dim=512,
    out_dim=512,
    hid_dim=256,
    num_layers=3,
    p_drop=0.05,
)

In [ ]:
train_ProbVLM(
    CLIP_Net,
    ProbVLM_Net,
    cub_train_loader,
    cub_valid_loader,
    Cri = TempCombLoss(),
    device='cuda',
    dtype=torch.cuda.FloatTensor,
    init_lr=8e-5,
    num_epochs=500,
    eval_every=5,
    ckpt_path='../ckpt/ProbVLM_Net',
    T1=1e0,
    T2=1e-4
)